# Does Logan Utah need more Parks? 

Data Science Capstone - IBM Data Science Professional Certificate on Coursera

## A description of the data

In 1914 Charles Downing Lay, a landscape architect for New York State, said that 12.5 percent of the total area of the city should be devoted to parks which is quite a lot of land devoted to parks and not really a realistic expectation.  By 1940 about one quarter of all cities having park facilities met the standard of one acre per 100 population.  Since that time the relationship between park acres and populations is no where near to that.  Most cities have recognize the standard of one acre per 100 population but their has been a lot of diversity of opinion concerning total open space.  This standard can not take into consideration all the different factors of each city such as cultural background, age, and socioeconomic status of the population. So it is hard to define how many acres to dedicate to a park.  

In this report to take into account these different factors and to simplify the question I look at different cities through out Utah and compare them to Logan Utah to see if Logan has enough parks compared to the rest of the state.  At the end I also see how many acres each city has compared to the standard one acre per 100 population.  This will give us insight to see if a new park needs to be built in the city of Logan Utah.

First off I needed to see how many parks that Logan has and where they are located.  To do this I used the foursquare API.  I defined a user_agent and set the address to Logan Utah.  I searched for a specific venue category that was "park".  So I was able to define the URL and send the get request.  This gave me a JSON file that I was able to clean and create a data frame.  Then I could map the data and see where the parks are.

Next I wanted to look at the rest of the cities in Utah to see how many parks they had, how many park acres, population and city acres.  I was able to get this information from three different place.  To get park number and park acreage I went to the web site https://opendata.gis.utah.gov/.  This web site is set up by Utah and has free data you can download.  I downloaded this in a .csv file.  I used pd.read_csv to read the csv file and put all the data in a data frame.

To get the population data I for the information I needed at the web site ,https://www.utah-demographics.com/cities_by_population.  I did the same thing as the park information.  I saved it as a .csv file and read it in to a panda data frame.  The total city acres came from census data from the web site https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2018_Gazetteer/2018_gaz_place_49.txt.  This was also in a .csv format and I read it into a panda data frame 

I had three different data frames that I needed to combine into one data frame so it would be easier to graph.  I did this using an inner join with the city as the index.  I dropped all the rows that had Nan meaning they didn't have parks because probably their small size. I filtered the data to look at cities that we around the same size as Logan.  I wanted to compare Logan to similar sized cities so I would get a better idea of how many park those cities had and if Logan was comparable.  This gave me a data frame I could work with and start to analyze the data.

### Foursquare API Park Data for Logan UT

### Installing and Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
import requests
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import seaborn as sns

import folium 
from bs4 import BeautifulSoup
import lxml
import requests # library to handle requests

import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::certifi-2019.11.28-py37_0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::certifi-2019.11.28-py37_0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::certifi-2019.11.28-py37_0
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::certifi-2019.11.28-py37_0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::certifi-2019.11.28-py37_0
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::certifi-2019.11.28-py37_0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::certifi-2019.11.28-py37_0, anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - anaconda

### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'RDLMTAMKGDD30NWQK3DLNRSAM35B3EYAXUCHNUJBNGSS3YOK' # your Foursquare ID
CLIENT_SECRET = 'K3AEMYFLWI32J5XHVKEKYYRZU3SN1AXDJITTPLU53HGQHW5F' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RDLMTAMKGDD30NWQK3DLNRSAM35B3EYAXUCHNUJBNGSS3YOK
CLIENT_SECRET:K3AEMYFLWI32J5XHVKEKYYRZU3SN1AXDJITTPLU53HGQHW5F


### Define a user_agent

In [3]:
address = 'Logan, Utah'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.7313447 -111.8348631


### Search for a specific venue category which in this case is for Parks

In [7]:
search_query = 'park'
radius = 4000
print(search_query + ' .... OK!')

park .... OK!


### Define the corresponding URL

In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=RDLMTAMKGDD30NWQK3DLNRSAM35B3EYAXUCHNUJBNGSS3YOK&client_secret=K3AEMYFLWI32J5XHVKEKYYRZU3SN1AXDJITTPLU53HGQHW5F&ll=41.7313447,-111.8348631&v=20180604&query=park&radius=4000&limit=30'

### Send the GET Request

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f15b6b5d1de386ab3a64de8'},
 'response': {'venues': [{'id': '531de09b498ee3958456ffc2',
    'name': 'Leetzow Business Park',
    'location': {'lat': 41.731306,
     'lng': -111.833449,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.731306,
       'lng': -111.833449}],
     'distance': 117,
     'cc': 'US',
     'city': 'Logan',
     'state': 'UT',
     'country': 'United States',
     'formattedAddress': ['Logan, UT', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d130941735',
      'name': 'Building',
      'pluralName': 'Buildings',
      'shortName': 'Building',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595258727',
    'hasPerk': False},
   {'id': '4be58b0bbcef2d7f58e703e5',
    'name': 'Merlin Olsen Park',
    'location': {'lat': 41.729900744747056,
     'lng': -111.828646789882,
     'labeledLatLngs': [

### Get relevant part of JSON and transform it into a pandas dataframe

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\Rachel Anderson\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.address,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood
0,531de09b498ee3958456ffc2,Leetzow Business Park,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",v-1595258727,False,41.731306,-111.833449,"[{'label': 'display', 'lat': 41.731306, 'lng':...",117,US,Logan,UT,United States,"[Logan, UT, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4be58b0bbcef2d7f58e703e5,Merlin Olsen Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1595258727,False,41.729901,-111.828647,"[{'label': 'display', 'lat': 41.72990074474705...",540,US,Logan,UT,United States,"[Logan, UT 84321, United States]",84321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51feaefd498e27af26488487,Kilowatt Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1595258727,False,41.725337,-111.843557,"[{'label': 'display', 'lat': 41.725337, 'lng':...",984,US,Logan,UT,United States,"[Logan, UT, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4bc0c6cb2a89ef3bf61ef188,Cache Valley Fun Park,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",v-1595258727,False,41.763370,-111.827191,"[{'label': 'display', 'lat': 41.76337031478518...",3621,US,North Logan,UT,United States,"[255 E 1770 N, North Logan, UT 84341, United S...",84341,255 E 1770 N,1894774,https://www.grubhub.com/restaurant/cache-valle...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN
4,5e52e19109326700070b6b25,Jones Neighborhood Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1595258727,False,41.744394,-111.845045,"[{'label': 'display', 'lat': 41.744394, 'lng':...",1680,US,Logan,UT,United States,"[Logan, UT 84321, United States]",84321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Define information of interest and filter dataframe

In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,address,neighborhood,id
0,Leetzow Business Park,Building,41.731306,-111.833449,"[{'label': 'display', 'lat': 41.731306, 'lng':...",117,US,Logan,UT,United States,"[Logan, UT, United States]",NaN,NaN,NaN,531de09b498ee3958456ffc2
1,Merlin Olsen Park,Park,41.729901,-111.828647,"[{'label': 'display', 'lat': 41.72990074474705...",540,US,Logan,UT,United States,"[Logan, UT 84321, United States]",84321,NaN,NaN,4be58b0bbcef2d7f58e703e5
2,Kilowatt Park,Park,41.725337,-111.843557,"[{'label': 'display', 'lat': 41.725337, 'lng':...",984,US,Logan,UT,United States,"[Logan, UT, United States]",NaN,NaN,NaN,51feaefd498e27af26488487
3,Cache Valley Fun Park,Playground,41.763370,-111.827191,"[{'label': 'display', 'lat': 41.76337031478518...",3621,US,North Logan,UT,United States,"[255 E 1770 N, North Logan, UT 84341, United S...",84341,255 E 1770 N,NaN,4bc0c6cb2a89ef3bf61ef188
4,Jones Neighborhood Park,Park,41.744394,-111.845045,"[{'label': 'display', 'lat': 41.744394, 'lng':...",1680,US,Logan,UT,United States,"[Logan, UT 84321, United States]",84321,NaN,NaN,5e52e19109326700070b6b25
5,Ryan's Place Park,None,41.722665,-111.820350,"[{'label': 'display', 'lat': 41.722665462048, ...",1545,US,River Heights,UT,United States,"[River Heights, UT 84321, United States]",84321,NaN,NaN,4e123bbafa769d21e9f4495c
6,Denzil Stewart Nature Park,Park,41.729170,-111.817631,"[{'label': 'display', 'lat': 41.72917, 'lng': ...",1451,US,Logan,UT,United States,"[Logan, UT 84321, United States]",84321,NaN,NaN,4ffe1bf8e4b026d5cb6c4eb7
7,Willow Park Zoo,Park,41.719251,-111.846789,"[{'label': 'display', 'lat': 41.71925126514661...",1671,US,Logan,UT,United States,"[419 W 700 S, Logan, UT 84321, United States]",84321,419 W 700 S,"Woodruff, lo",4bcdf861b6c49c74dc4b9691
8,Skate Park,Skate Park,41.723931,-111.850876,"[{'label': 'display', 'lat': 41.72393094400192...",1565,US,Logan,UT,United States,"[400-498 S 700 W, Logan, UT 84321, United States]",84321,400-498 S 700 W,NaN,4e1680df7d8b69590f4369db
9,Adams Park,Park,41.741217,-111.822184,"[{'label': 'display', 'lat': 41.74121708790623...",1522,US,Logan,UT,United States,"[500 N 500 E, Logan, UT 84321, United States]",84321,500 N 500 E,"Adams, Logan, UT",4c32614fed37a5936dc86b03


### Map points

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=14) 

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Data for all other cities in Utah to compare Population, City Acres, Number of Parks, and Park Acres.
### Population Data from ,https://www.utah-demographics.com/cities_by_population
### Number of parks and park acres  https://opendata.gis.utah.gov/ 
### City square mile converted to acres from https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2018_Gazetteer/2018_gaz_place_49.txt

In [13]:
park_by_city = pd.read_csv('Desktop\ParksCount.csv',sep=';')
population_by_city = pd.read_csv('Desktop\population.csv',sep=';')
acres_by_city = pd.read_csv('Desktop\cityacres.csv',sep=';')

### Combine dataframes and drop rows with NaN 

In [14]:
total_df = population_by_city.set_index('city').join(park_by_city.set_index('city'))
total_df = total_df.join(acres_by_city.set_index('city'))
total_df.dropna(inplace=True)

total_df

,population,park_acres,parks,sqmile,lat,long,total_acres,geoid
city,,,,,,,,
Alpine,10498,178.431851,21.0,7.952,40.462942,-111.772433,5089.28,4900540.0
Amalga,558,5.552086,1.0,3.485,41.856274,-111.897145,2230.40,4901090.0
Antimony,121,2.765945,1.0,9.823,38.103019,-111.982295,6286.72,4901860.0
Aurora,1052,12.547009,3.0,1.039,38.920204,-111.934021,664.96,4902740.0
Ballard,1093,14.445762,1.0,13.980,40.294842,-109.949362,8947.20,4903345.0
Bear River City,898,11.565654,1.0,1.797,41.612377,-112.124953,1150.08,4903950.0
Beaver,3185,59.180506,5.0,6.681,38.275931,-112.638298,4275.84,4904060.0
Bicknell,336,14.845345,3.0,0.571,38.340667,-111.544159,365.44,4905490.0
Blanding,3633,48.563296,11.0,13.207,37.623143,-109.512951,8452.48,4906370.0


### Filter so we are only looking at simlar sized cities

In [15]:
total_filtered_df = total_df.query('10000 <= population <= 200000')
total_filtered_df

,population,park_acres,parks,sqmile,lat,long,total_acres,geoid
city,,,,,,,,
Alpine,10498,178.431851,21.0,7.952,40.462942,-111.772433,5089.28,4900540.0
Bluffdale,16358,153.027326,5.0,10.979,40.474551,-111.940262,7026.56,4906810.0
Bountiful,43981,82.464873,8.0,13.204,40.872483,-111.861200,8450.56,4907690.0
Brigham City,19601,92.762486,11.0,24.511,41.505904,-112.051418,15687.04,4908460.0
Cedar City,34764,66.396818,9.0,35.860,37.682874,-113.095539,22950.40,4911320.0
Cedar Hills,10083,26.208535,7.0,2.718,40.412028,-111.754266,1739.52,4911440.0
Centerville,17587,40.523935,5.0,5.973,40.926297,-111.883339,3822.72,4911980.0
Clearfield,32118,65.419434,8.0,7.713,41.097049,-112.017959,4936.32,4913850.0
Clinton,22499,49.286532,8.0,5.932,41.138740,-112.066417,3796.48,4914290.0
